Import

In [2]:
pip install numpy-stl

Package               Version
--------------------- ----------------
apturl                0.5.2
asttokens             2.4.1
bcrypt                3.2.0
blinker               1.4
Brlapi                0.8.3
certifi               2020.6.20
chardet               4.0.0
click                 8.0.3
colorama              0.4.4
comm                  0.2.2
command-not-found     0.3
contourpy             1.2.1
cryptography          3.4.8
cupshelpers           1.0
cycler                0.12.1
dbus-python           1.2.18
debugpy               1.8.1
decorator             5.1.1
defer                 1.0.6
distro                1.7.0
distro-info           1.1+ubuntu0.2
duplicity             0.8.21
et-xmlfile            1.1.0
exceptiongroup        1.2.1
executing             2.0.1
fasteners             0.14.1
fonttools             4.51.0
future                0.18.2
httplib2              0.20.2
idna                  3.3
importlib-metadata    4.6.4
ipykernel             6.29.4
ipython               8

In [5]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from stl import mesh

In [15]:
def find_data(path : str) -> list:

    files = os.listdir(path)
    data = [None for _ in range(len(files))]

    with tqdm(total=len(files)) as pbar:
        for file in files:
            file_path = os.path.join(path, file)
            name, entention = os.path.splitext(file)
            data[int(name)-1] = np.array(pd.read_excel(file_path).values, dtype=np.float64)
            pbar.update(1)
    
    return data

In [20]:
data = find_data("/home/corn/Documents/ScienceFair/Data/Data_2/Cubic/4")[:-1]

num_lines = len(data)

data_limit = min([max([j[5] for j in i]) for i in data]) # 1:時間 2:位移 3:力量 4:應力 5:應變

initarray = np.arange(0, data_limit, 0.005)

data = [np.interp(initarray, d[:,5], d[:,4]) for d in data]

print(np.shape(data))

100%|██████████| 104/104 [00:08<00:00, 12.18it/s]

(103, 499)


Draw Stl

In [18]:
# 點

vertices = []

for i in range(num_lines):
    for j in range(len(initarray)):
        x = initarray[j]
        y = i * data_limit / num_lines
        z = data[i][j]
        vertices.append([x, y, z])

# 四角

vertices.append([initarray[0], 0, 0]) # -4 rd
vertices.append([initarray[-1], 0, 0]) # -3 rt
vertices.append([initarray[0], data_limit - 0.025, 0]) # -2 ld
vertices.append([initarray[-1], data_limit - 0.025, 0]) # -1 lt

vertices = np.array(vertices)

In [21]:
# 面

faces = []

for i in range(num_lines-1):
    i *= len(initarray)
    for j in range(len(initarray)-1):
        p1 = i + j
        p2 = i + j + 1
        p3 = i + j + len(initarray)
        faces.append([p1, p2, p3])
        p1 = i + j + len(initarray)
        p2 = i + j + len(initarray) + 1
        p3 = i + j + 1
        faces.append([p1, p2, p3])

# 底

faces.append([len(vertices)-1, len(vertices)-2, len(vertices)-3])
faces.append([len(vertices)-4, len(vertices)-2, len(vertices)-3])

# 側

faces.append([len(vertices)-4, len(vertices)-3, 0])
faces.append([len(vertices)-2, len(vertices)-1, (num_lines-1) * len(initarray)])
for i in range(len(initarray)-1):
    p1 = len(vertices)-3
    p2 = i
    p3 = i + 1
    faces.append([p1, p2, p3])
    p1 = len(vertices)-1
    p2 = (num_lines - 1) * len(initarray) + i
    p3 = (num_lines - 1) * len(initarray) + i + 1
    faces.append([p1, p2, p3])
    
# 前後

faces.append([len(vertices)-4, len(vertices)-2, 0])
faces.append([len(vertices)-3, len(vertices)-1, num_lines * len(initarray) - 1])
for i in range(num_lines-1):
    i *= len(initarray)
    p1 = len(vertices)-2
    p2 = i
    p3 = i + len(initarray)
    faces.append([p1, p2, p3])
    p1 = len(vertices)-3
    p2 = len(initarray) + i - 1
    p3 = 2*len(initarray) + i - 1
    faces.append([p1, p2, p3])

faces = np.array(faces)

In [22]:
# 創建STL網格對象
cube_mesh = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))
for i, f in enumerate(faces):
    for j in range(3):
        cube_mesh.vectors[i][j] = vertices[f[j], :]

# 將STL網格保存為檔案
cube_mesh.save('/home/corn/Documents/ScienceFair/Test2/Output/output.stl')